<a href="https://colab.research.google.com/github/gulliyevn/MachineLearningProjects/blob/main/CaliforniaHosuing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!wget https://raw.githubusercontent.com/Rstam59/TaskDataRepoForStudents/main/housing.csv

In [ ]:
hs=pd.read_csv('/content/housing.csv')
hs

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [ ]:
hs.hist(bins=50,figsize=(12,7));

In [ ]:
hs.corr(numeric_only=True)['median_house_value'].sort_values(ascending=True)

In [ ]:
plt.scatter(hs['longitude'],hs['latitude'],alpha=0.2);

In [ ]:
hs.plot(kind='scatter',
        x=['longitude'],
        y=['latitude'],
        c='median_house_value',
        cmap='jet',
        grid=True,
        s=hs['population']/100,
        legend=True,
        label='House',
        figsize=(12,7));

In [ ]:
from pandas.plotting import scatter_matrix

attributes=['total_bedrooms', 'population', 'median_income','median_house_value',]
scatter_matrix(hs[attributes],figsize=(12,8));

In [ ]:
plt.scatter(hs['median_income'],hs['median_house_value']);

In [ ]:
hs['income_cat']=pd.cut(hs['median_income'],
                    bins=[0.,1.5,3.0,4.5,6.0,np.inf],
                        labels=[1,2,3,4,5])

hs['income_cat'].hist();

In [ ]:
hs['rooms_per_house']=hs['total_rooms']/hs['households']
hs['bedrooms_ratios']=hs['total_bedrooms']/hs['total_rooms']
hs['people_per_house']=hs['population']/hs['households']

In [ ]:
hs.corr(numeric_only=True)['median_house_value'].sort_values(ascending=False)

In [ ]:
attributes=['median_house_value','bedrooms_ratios','people_per_house','rooms_per_house']
scatter_matrix(hs[attributes],figsize=(10,7));

In [ ]:
np.log(hs['total_bedrooms']).hist(bins=50)

In [ ]:
hs['total_bedrooms'].apply(np.log)

In [ ]:
X=hs.drop('median_house_value',axis=1)
y=hs['median_house_value'].copy()

In [ ]:
np.random.seed(42)
X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=hs['income_cat'],test_size=0.2)
X_train

In [ ]:
X_train.drop('income_cat',axis=1,inplace=True)
X_test.drop('income_cat',axis=1,inplace=True)

In [ ]:
txt=X_test.select_dtypes(exclude=[np.number]).columns
num=X_test.select_dtypes(include=[np.number]).columns

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
txtpip=Pipeline([('impute',SimpleImputer(strategy='constant',fill_value='most_frequent')),
                  ('onehot',OneHotEncoder())])
numpip=Pipeline([('impute',SimpleImputer(strategy='median')),
                 ('standart_scaler',StandardScaler())
                 ])

In [ ]:
transformer=ColumnTransformer([('onehotpip',txtpip,txt),
                               ('onehotnum',numpip,num)],remainder='passthrough')

In [ ]:
Xtt=transformer.fit_transform(X_train)
tc=transformer.get_feature_names_out()
Xttdf=pd.DataFrame(Xtt)

In [ ]:
Xtst=transformer.transform(X_test)
Xtstdf=pd.DataFrame(Xtst)

In [ ]:
reg=RandomForestRegressor()

In [ ]:
reg.fit(Xttdf,y_train)

In [ ]:
reg.score(Xtstdf,y_test)

In [ ]:
np.random.seed(42)
model = RandomForestRegressor()

full_pipeline = Pipeline([
('preprocessing', transformer),
('random_forest', model)])

full_pipeline.fit(X_train, y_train)

In [ ]:
full_pipeline.score(X_test, y_test)

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(full_pipeline,X,y,cv=5)

In [ ]:
from sklearn.metrics import mean_absolute_error

preds=full_pipeline.predict(X_test)

preds

In [ ]:
mean_absolute_error(preds,y_test)

In [ ]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()
full_pipeline1 = Pipeline([
('preprocessing', transformer),
('random_forest', model)])

full_pipeline1.fit(X_train, y_train)

preds1=full_pipeline1.predict(X_test)

preds1



In [ ]:
full_pipeline1.score(X_test, y_test)

In [ ]:
mean_absolute_error(preds1,y_test)

In [ ]:
from sklearn.svm import SVR
model=SVR()
full_pipeline2 = Pipeline([
('preprocessing', transformer),
('random_forest', model)])

full_pipeline2.fit(X_train, y_train)

preds2=full_pipeline2.predict(X_test)

preds2


In [ ]:
full_pipeline2.score(X_test, y_test)

In [ ]:
mean_absolute_error(preds2,y_test)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model=DecisionTreeRegressor()
full_pipeline3 = Pipeline([
('preprocessing', transformer),
('random_forest', model)])

full_pipeline3.fit(X_train, y_train)

preds3=full_pipeline3.predict(X_test)

preds3

In [ ]:
full_pipeline3.score(X_test, y_test)

In [ ]:
mean_absolute_error(preds3,y_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model=RandomForestRegressor()

full_pipeline4 = Pipeline([
('preprocessing', transformer),
('random_forest', model)])

full_pipeline4.fit(X_train, y_train)

preds4=full_pipeline4.predict(X_test)

preds4

array([392742.1 , 206209.01, 410074.26, ..., 250688.  , 186174.  ,
       269658.06])

In [ ]:
full_pipeline4.score(X_test, y_test)

0.809236544564998

In [ ]:
mean_absolute_error(preds4,y_test)

32638.168553779076

Fine-Tunning

In [ ]:
# for i in range (50,200,10):
#   rr=RandomForestRegressor(n_estimators=i)
#   rr.fit(Xttdf,y_train)
#   score=rr.score(Xtstdf,y_test)
#   print(score)

In [107]:
from sklearn.model_selection import GridSearchCV
model=RandomForestRegressor()

param_grid={
    'n_estimators':[50,60],
    'max_depth':[10,15,30]

}

grid_search=GridSearchCV(estimator=model,param_grid=param_grid,cv=3)
grid_search.fit(Xttdf,y_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [10, 15, 30], 'n_estimators': [50, 60]})

In [108]:
grid_search.score(Xtstdf,y_test)

0.8086639979584785

In [109]:
grid_search.best_score_

0.805289247059616

In [110]:
grid_search.best_estimator_

RandomForestRegressor(max_depth=30, n_estimators=60)

In [111]:
grid_search.best_params_

{'max_depth': 30, 'n_estimators': 60}

In [112]:
from sklearn.tree import DecisionTreeRegressor
model=DecisionTreeRegressor()

param_grid ={
    'max_depth':[10,15,30],
    'max_features':['sqrt','log2'],
    'splitter':['best','random']
}

tree=DecisionTreeRegressor()
reg=GridSearchCV(tree,param_grid=param_grid)
reg.fit(Xttdf,y_train)

GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [10, 15, 30],
                         'max_features': ['sqrt', 'log2'],
                         'splitter': ['best', 'random']})

In [113]:
reg.best_params_

{'max_depth': 10, 'max_features': 'log2', 'splitter': 'best'}

In [114]:
reg.score(Xtstdf,y_test)

0.6683136080195371

In [115]:
reg.best_estimator_.score(Xtstdf,y_test)

0.6683136080195371

In [116]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()

param_grid ={
    'copy_X':[True,False],
    'fit_intercept':[True,False],
    'n_jobs':[-1]
}

lin_reg=GridSearchCV(model,param_grid=param_grid)
lin_reg.fit(Xttdf,y_train)

GridSearchCV(estimator=LinearRegression(),
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False], 'n_jobs': [-1]})

In [117]:
lin_reg.best_estimator_

LinearRegression(n_jobs=-1)

In [118]:
lin_reg.best_score_

0.6581618216571978

In [119]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
txtpip=Pipeline([('impute',SimpleImputer(strategy='constant',fill_value='most_frequent')),
                  ('onehot',OneHotEncoder())])
numpip=Pipeline([('impute',SimpleImputer(strategy='median')),
                 ('standart_scaler',StandardScaler())
                 ])
transformer=ColumnTransformer([('onehotpip',txtpip,txt),
                               ('onehotnum',numpip,num)],remainder='passthrough')


full_pipeline = Pipeline([
    ('preprocessing', transformer),
     ('model', LinearRegression())])


param_grid ={
    'model__copy_X':[True,False],
    'model__fit_intercept':[True,False],
    'model__n_jobs':[-1]
}

lin_reg=GridSearchCV(full_pipeline,param_grid)
lin_reg.fit(X_train,y_train)

GridSearchCV(estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehotpip',
                                                                         Pipeline(steps=[('impute',
                                                                                          SimpleImputer(fill_value='most_frequent',
                                                                                                        strategy='constant')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder())]),
                                                                         Index(['ocean_proximity'], dtype='object')),
                                                                        ('onehotnum',
                                                                         Pipeline(steps=[('impute',
                                                                                          SimpleImputer(strategy='m...
                                                                                          StandardScaler())]),
                                                                         Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'rooms_per_house', 'bedrooms_ratios', 'people_per_house'],
      dtype='object'))])),
                                       ('model', LinearRegression())]),
             param_grid={'model__copy_X': [True, False],
                         'model__fit_intercept': [True, False],
                         'model__n_jobs': [-1]})

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler,StandardScaler
txtpip=Pipeline([('impute',SimpleImputer(strategy='constant',fill_value='most_frequent')),
                  ('onehot',OneHotEncoder())])
numpip=Pipeline([('impute',SimpleImputer(strategy='median')),
                 ('standart_scaler',StandardScaler())
                 ])
transformer=ColumnTransformer([('onehotpip',txtpip,txt),
                               ('onehotnum',numpip,num)],remainder='passthrough')


full_pipeline = Pipeline([
    ('preprocessing', transformer),
     ('model', LinearRegression())])


param_grid ={
    'model__copy_X':[True,False],
    'model__fit_intercept':[True,False]
}

lin_reg=RandomizedSearchCV(full_pipeline,param_grid
                           )
lin_reg.fit(X_train,y_train)

In [ ]:
lin_reg.score(X_test,y_test)